# 1. Load The Libraries : 

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# 2. Read The Images Path: 

In step 2, all the path for the the images were read and I create a dataframe with two columns which are : 1. path 2.label

In [ ]:
path = 'D:/other/python/IMT/Breast Cancer/Breast-Cancer/Data'

categories = [['benign',438],['malignant',211],['normal',134]]

images_path_list= []

all_image_path_list = []

for i in range(len(categories)) : 
    
    new_path = (f'{path}/{categories[i][0]}')
    
    for j in range(1,categories[i][1]):
    
        image_path = (f'{new_path}/{categories[i][0]} ({j}).png') 
        
        images_path_list.append(image_path)
        
    image_path_df = pd.DataFrame()
    
    image_path_df['Path'] =   images_path_list
    image_path_df['Label'] = categories[i][0]
    
    all_image_path_list.append(image_path_df)
    
    images_path_list = []
    
image_path_df = pd.concat(all_image_path_list,axis = 0)
image_path_df.reset_index(drop=True,inplace=True)
        
image_path_df.to_csv('D:/other/python/IMT/Breast Cancer/Breast-Cancer/Data/path.csv')
image_path_df

# 3. Shuffle Dataset : 

I notice that there is imbalance in dataset, because the number of the avalaible samples for each label in not the same, so I decided to consider the value 133 for each label that in the minimum number of the sample for lables.

Then, I shuffle all dataset, because the pathes were inported in order, so I shuffle them.


In [ ]:
benign_df = image_path_df.iloc[0:133,:]

malignant_df = image_path_df.iloc[437:646,:]

normal_df = image_path_df.iloc[647:780,:]

shuffled_benign_df = benign_df.sample(133)

shuffled_malignant_df = malignant_df.sample(133)

final_df = pd.concat([shuffled_benign_df,shuffled_malignant_df,normal_df],axis=0)

final_df.reset_index(drop=True,inplace=True)

final_df

final_df['Label'].value_counts()

# 4. Read The Images : 

In step 4, I read the images from the pathes and also resize them, bvecause I need to be in the same size to generate a vector in the same size and then spilit data to the training and test to feed the model.

In [ ]:
X = final_df['Path']
Y = final_df['Label'].replace({'benign':0,'malignant':1,'normal':2}).values.reshape(-1,1)

def read_image(image_pathes):
    
    width,height = 400,400
    final_array = np.empty((480000,),int)
      
    for image_path in image_pathes : 
        
        image = cv2.imread(image_path)
        
        resized_image = cv2.resize(image,(width,height))
        
        flatted_image = resized_image.flatten()
        
        final_array = np.vstack((final_array,flatted_image))
        
        
    final_array = np.delete(final_array,0,axis=0)    
    
    return final_array

readable_X = read_image(X)  

X_train,X_test,Y_train,Y_test = train_test_split(readable_X,Y,test_size=0.2,random_state=0)


# 5. Define Decision Tree Model : 

finally I define a function to create the final model and calculate the final accuracy with the respect to the different attributes that are avalaible in Decision Tree model in skeat learn and save the final result.

In [ ]:
def Decision_Tree():
    
    criterion = ["gini", "entropy"]

    max_depth = [None,3,4,5,6,7,8,9,10]
    
    initial_accuracy = 0

    for i in criterion : 
             
        for j in max_depth : 
            
            DecisionTree_Model = DecisionTreeClassifier(criterion=i,max_depth=j)
            
            DecisionTree_Model.fit(X=X_train,y=Y_train.ravel())
            
            Y_prediction = DecisionTree_Model.predict(X=X_test)
            
            model_accuracy = accuracy_score(y_true=Y_test,y_pred=Y_prediction)
            
            if model_accuracy > initial_accuracy : 
                
                initial_accuracy = model_accuracy
                final_model = DecisionTree_Model
                final_Y_prdiction = Y_prediction
                final_criterion = i
                final_MaxDepth = j
                print(f'This current model with criterion = {final_criterion}, max_depth = {final_MaxDepth} has the {initial_accuracy*100}% accuracy.')
                print(f'\n--------------------------------------------------------------------------------------------------------------------------\n')
                    
    print(f'This final model with criterion = {final_criterion}, max_depth = {final_MaxDepth} has the {initial_accuracy*100}% accuracy.') 
    
    return final_model,final_Y_prdiction

# 6. Decision Tree Model : 

In [ ]:
DecisionTree_model,Y_prediction_DTM = Decision_Tree()

# 7. Define A Function To Evalute The Model : 

In [ ]:
def evaluate(Y_prediction,name):
    
    report = classification_report(y_true=Y_test,y_pred=Y_prediction)
    
    print(report)
    
    print('\n---------------------------------------------------------------------------------------------------------------\n')
    
    conf_matrix = confusion_matrix(y_true=Y_test,y_pred=Y_prediction)
    
    plt.figure(figsize=(8,5))
    
    sns.heatmap(conf_matrix,annot=True,cmap='Blues')
    
    plt.xlabel('Predicted Label')
    
    plt.ylabel('True Label')
    
    plt.title(f'{name} Evaluation')
    
    plt.show()

# 8. Evaluate The Model : 

In [ ]:
evaluate(Y_prediction=Y_prediction_DTM,name='Decision-Tree')

# 9. Random Forest Function : 

In [ ]:
def Random_Forest():
    
    estimator = [10,20,30,40,50,60,70,80,90,100]

    initial_accuracy = 0

    for i in estimator:
        
        model = RandomForestClassifier(n_estimators=i,criterion='gini',max_depth=8)
        
        model.fit(X=X_train,y=Y_train.ravel())
        
        Y_prediction = model.predict(X=X_test)
        
        model_accuracy = accuracy_score(y_true=Y_test,y_pred=Y_prediction)
        
        if model_accuracy > initial_accuracy : 
            
            final_model = model
            initial_accuracy = model_accuracy
            final_Y_prediction = Y_prediction
            final_estimator = i
            print(f'This current model with estimator = {final_estimator}, has the {initial_accuracy*100}% accuracy.')
            print(f'\n------------------------------------------------------------------------------------------------------------------\n')               
    
    print(f'The final model with estimator = {final_estimator}, has the {initial_accuracy*100}% accuracy.')
    
    return final_model,final_Y_prediction

# 10. Random Forest Model :

In [ ]:
RandomForest_Model,Y_prediction_RFM = Random_Forest()

# 11. Evaluate The Model : 

In [ ]:
evaluate(Y_prediction=Y_prediction_RFM,name='Random-Forest')

# 12. SVM Model Function : 

In [ ]:
def SVM():    
    grid = {
        'C' : [0.001,0.01,0.1,1,10],
        
        'kernel' : ["linear","poly","rbf","sigmoid"],
        
        'degree' : [1,3,5,7],
        
        'gamma' : [0.01,0.1,1]
    }

    SVM_Model = SVC()

    SVM_CV = GridSearchCV(estimator=SVM_Model,param_grid=grid,cv=5)

    SVM_CV.fit(X=X_train,y=Y_train.ravel())

    print(f'\nThe best parameters are : {SVM_CV.best_params_}\n------------------------------------------------------------------------------\n')
    print(f'\nThe train accuracy is equal to : {SVM_CV.best_score_}\n------------------------------------------------------------------------------\n')
    print(f'\nThe test accuracy is equal to : {SVM_CV.score(X=X_test,y=Y_test)}\n------------------------------------------------------------------------------\n')
    
    return SVM_CV.best_estimator_

# 13. SVM Model : 

In [ ]:
SVM_Model = SVM()